# Hurricane Data Exploration

This notebook demonstrates how to load and explore hurricane data using the Hurricane Forecast AI system.

## Contents
1. Setup and imports
2. Load HURDAT2 data
3. Explore hurricane tracks
4. Load ERA5 reanalysis data
5. Visualize hurricane data
6. Data preprocessing examples

## 1. Setup and Imports

In [ ]:
# Standard imports
import sys
import warnings
from pathlib import Path
warnings.filterwarnings('ignore')

# Add project root to path
project_root = Path('.').absolute().parent
sys.path.append(str(project_root))

# Data manipulation
import numpy as np
import pandas as pd
import xarray as xr

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

# Hurricane forecast imports
from src.data.loaders import HURDAT2Loader, IBTrACSLoader, ERA5Loader, HurricaneDataPipeline
from src.data.processors import HurricanePreprocessor, ERA5Preprocessor
from src.data.validators import HurricaneDataValidator
from src.utils import setup_logging, get_config

# Set up logging
setup_logging(level='INFO')

# Set style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')

## 2. Load HURDAT2 Data

In [ ]:
# Initialize HURDAT2 loader
hurdat2_loader = HURDAT2Loader()

# Load data (will download if not present)
storms_df, tracks_df = hurdat2_loader.load()

print(f"Loaded {len(storms_df)} storms with {len(tracks_df)} total track points")
print(f"\nData spans from {tracks_df['timestamp'].min()} to {tracks_df['timestamp'].max()}")

In [ ]:
# Examine storm data
print("Storm data columns:")
print(storms_df.columns.tolist())
print("\nSample storms:")
storms_df.head()

In [ ]:
# Examine track data
print("Track data columns:")
print(tracks_df.columns.tolist())
print("\nSample track points:")
tracks_df.head()

In [ ]:
# Storm statistics by year
storm_counts = storms_df.groupby('year').size()

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

# Number of storms per year
storm_counts.plot(ax=ax1, color='navy')
ax1.set_title('Number of Storms per Year')
ax1.set_xlabel('Year')
ax1.set_ylabel('Number of Storms')
ax1.grid(True, alpha=0.3)

# Hurricane intensity distribution
intensity_data = tracks_df[tracks_df['max_wind'].notna()]['max_wind']
ax2.hist(intensity_data, bins=30, color='darkred', alpha=0.7, edgecolor='black')
ax2.set_title('Maximum Wind Speed Distribution')
ax2.set_xlabel('Maximum Wind Speed (knots)')
ax2.set_ylabel('Frequency')
ax2.axvline(64, color='orange', linestyle='--', label='Hurricane threshold')
ax2.legend()

plt.tight_layout()
plt.show()

## 3. Explore Hurricane Tracks

In [ ]:
# Select a specific hurricane to analyze
# Example: Hurricane Katrina (2005)
katrina = hurdat2_loader.get_storm('AL122005')

print(f"Hurricane Katrina track points: {len(katrina)}")
print(f"Duration: {katrina['timestamp'].min()} to {katrina['timestamp'].max()}")
print(f"Peak intensity: {katrina['max_wind'].max()} knots")
print(f"Minimum pressure: {katrina['min_pressure'].min()} mb")

In [ ]:
# Plot hurricane track with intensity
fig = plt.figure(figsize=(12, 8))
ax = plt.axes(projection=ccrs.PlateCarree())

# Add map features
ax.add_feature(cfeature.LAND, alpha=0.5)
ax.add_feature(cfeature.OCEAN, alpha=0.3)
ax.add_feature(cfeature.COASTLINE)
ax.add_feature(cfeature.BORDERS, alpha=0.5)
ax.add_feature(cfeature.STATES, alpha=0.5)

# Plot track colored by intensity
track = ax.scatter(
    katrina['longitude'], 
    katrina['latitude'],
    c=katrina['max_wind'],
    s=50,
    cmap='YlOrRd',
    transform=ccrs.PlateCarree(),
    alpha=0.8,
    edgecolors='black',
    linewidths=0.5
)

# Plot track line
ax.plot(
    katrina['longitude'],
    katrina['latitude'],
    'k-',
    transform=ccrs.PlateCarree(),
    alpha=0.5,
    linewidth=1
)

# Set extent
ax.set_extent([-100, -65, 15, 35], crs=ccrs.PlateCarree())

# Add colorbar
cbar = plt.colorbar(track, ax=ax, pad=0.05)
cbar.set_label('Maximum Wind Speed (knots)', rotation=270, labelpad=20)

# Add title and grid
ax.set_title('Hurricane Katrina (2005) Track and Intensity', fontsize=14, fontweight='bold')
ax.gridlines(draw_labels=True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Analyze multiple major hurricanes
major_hurricanes = [
    ('AL092017', 'Harvey', 2017),
    ('AL112017', 'Irma', 2017),
    ('AL152017', 'Maria', 2017),
    ('AL142018', 'Michael', 2018),
    ('AL062019', 'Dorian', 2019)
]

fig = plt.figure(figsize=(15, 10))
ax = plt.axes(projection=ccrs.PlateCarree())

# Add map features
ax.add_feature(cfeature.LAND, alpha=0.5)
ax.add_feature(cfeature.OCEAN, alpha=0.3)
ax.add_feature(cfeature.COASTLINE)
ax.add_feature(cfeature.BORDERS, alpha=0.5)

# Plot each hurricane
for storm_id, name, year in major_hurricanes:
    try:
        track = hurdat2_loader.get_storm(storm_id)
        ax.plot(
            track['longitude'],
            track['latitude'],
            '-',
            transform=ccrs.PlateCarree(),
            linewidth=2,
            label=f'{name} ({year})',
            alpha=0.8
        )
    except Exception as e:
        print(f"Could not load {name}: {e}")

# Set extent for Atlantic basin
ax.set_extent([-100, -20, 10, 50], crs=ccrs.PlateCarree())

# Add legend and title
ax.legend(loc='lower left', fontsize=10)
ax.set_title('Major Atlantic Hurricanes (2017-2019)', fontsize=16, fontweight='bold')
ax.gridlines(draw_labels=True, alpha=0.3)

plt.tight_layout()
plt.show()

## 4. Data Preprocessing and Validation

In [ ]:
# Initialize preprocessor and validator
preprocessor = HurricanePreprocessor()
validator = HurricaneDataValidator()

# Preprocess Katrina data
katrina_processed = preprocessor.normalize_track_data(katrina)
katrina_features = preprocessor.create_track_features(katrina_processed)

print("New features created:")
new_columns = set(katrina_features.columns) - set(katrina.columns)
print(sorted(new_columns))

In [ ]:
# Validate the track data
validation_results = validator.validate_track(katrina)

print(f"Validation passed: {validation_results['valid']}")
if validation_results['errors']:
    print("\nErrors:")
    for error in validation_results['errors']:
        print(f"  - {error}")
if validation_results['warnings']:
    print("\nWarnings:")
    for warning in validation_results['warnings']:
        print(f"  - {warning}")

In [ ]:
# Visualize preprocessed features
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
axes = axes.ravel()

# Storm motion speed
axes[0].plot(katrina_features['timestamp'], katrina_features['storm_speed'], 'b-', linewidth=2)
axes[0].set_title('Storm Motion Speed')
axes[0].set_ylabel('Speed (knots)')
axes[0].grid(True, alpha=0.3)

# Intensity change
axes[1].plot(katrina_features['timestamp'], katrina_features['intensity_change'], 'r-', linewidth=2)
axes[1].axhline(0, color='black', linestyle='--', alpha=0.5)
axes[1].axhline(30, color='orange', linestyle='--', alpha=0.5, label='Rapid intensification')
axes[1].set_title('Intensity Change (6-hourly)')
axes[1].set_ylabel('Wind Speed Change (knots)')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

# Pressure deficit
axes[2].plot(katrina_features['timestamp'], katrina_features['pressure_deficit'], 'g-', linewidth=2)
axes[2].set_title('Pressure Deficit')
axes[2].set_ylabel('Pressure Deficit (mb)')
axes[2].grid(True, alpha=0.3)

# Wind-Pressure relationship
axes[3].scatter(
    katrina_features['pressure_deficit'],
    katrina_features['max_wind'],
    c=katrina_features['day_of_year'],
    cmap='viridis',
    s=50,
    alpha=0.7
)
axes[3].set_title('Wind-Pressure Relationship')
axes[3].set_xlabel('Pressure Deficit (mb)')
axes[3].set_ylabel('Maximum Wind (knots)')
axes[3].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 5. Prepare Training Sequences

In [ ]:
# Create sequences for model training
sequence_length = 8  # 48 hours of history (6-hourly data)
forecast_length = 20  # 120 hours forecast

try:
    inputs, targets = preprocessor.prepare_sequences(
        katrina_features,
        sequence_length=sequence_length,
        forecast_length=forecast_length
    )
    
    print(f"Input sequences shape: {inputs.shape}")
    print(f"Target sequences shape: {targets.shape}")
    print(f"Number of training samples: {len(inputs)}")
    print(f"\nFeatures used: {preprocessor.feature_names}")
    
except ValueError as e:
    print(f"Error creating sequences: {e}")
    print("Track may be too short for the specified sequence lengths")

## 6. ERA5 Data Integration (Optional)

In [ ]:
# Check if ERA5 sample data exists
config = get_config()
era5_sample_path = Path(config.data.era5_cache_dir) / "era5_sample_2023_hurricane_lee.nc"

if era5_sample_path.exists():
    print("Loading ERA5 sample data...")
    era5_loader = ERA5Loader()
    era5_data = xr.open_dataset(era5_sample_path)
    
    print(f"ERA5 data shape: {dict(era5_data.dims)}")
    print(f"Variables: {list(era5_data.data_vars)}")
    print(f"Time range: {era5_data.time.min().values} to {era5_data.time.max().values}")
    
    # Visualize sample ERA5 field
    if 'msl' in era5_data:
        fig = plt.figure(figsize=(12, 8))
        ax = plt.axes(projection=ccrs.PlateCarree())
        
        # Plot mean sea level pressure
        time_idx = len(era5_data.time) // 2  # Middle time
        msl = era5_data['msl'].isel(time=time_idx) / 100  # Convert to mb
        
        im = ax.contourf(
            era5_data.longitude,
            era5_data.latitude,
            msl,
            levels=20,
            cmap='RdBu_r',
            transform=ccrs.PlateCarree()
        )
        
        # Add contour lines
        cs = ax.contour(
            era5_data.longitude,
            era5_data.latitude,
            msl,
            levels=10,
            colors='black',
            linewidths=0.5,
            transform=ccrs.PlateCarree()
        )
        ax.clabel(cs, inline=True, fontsize=8)
        
        # Add map features
        ax.add_feature(cfeature.COASTLINE)
        ax.add_feature(cfeature.BORDERS, alpha=0.5)
        
        # Add colorbar and title
        cbar = plt.colorbar(im, ax=ax, pad=0.05)
        cbar.set_label('Pressure (mb)', rotation=270, labelpad=20)
        
        time_str = pd.Timestamp(era5_data.time.isel(time=time_idx).values).strftime('%Y-%m-%d %H:%M')
        ax.set_title(f'ERA5 Mean Sea Level Pressure - {time_str}', fontsize=14)
        ax.gridlines(draw_labels=True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()
else:
    print("ERA5 sample data not found.")
    print("Run 'python scripts/setup_data.py --download-era5' to download sample data.")

## 7. Summary Statistics

In [ ]:
# Analyze hurricanes by decade
tracks_df['decade'] = (tracks_df['timestamp'].dt.year // 10) * 10
hurricanes_only = tracks_df[tracks_df['max_wind'] >= 64]

decade_stats = hurricanes_only.groupby(['storm_id', 'decade'])['max_wind'].max().reset_index()
decade_summary = decade_stats.groupby('decade')['max_wind'].agg(['count', 'mean', 'max'])

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

# Number of hurricanes by decade
decade_summary['count'].plot(kind='bar', ax=ax1, color='darkblue', alpha=0.7)
ax1.set_title('Number of Hurricanes by Decade')
ax1.set_xlabel('Decade')
ax1.set_ylabel('Number of Hurricanes')
ax1.grid(True, alpha=0.3)

# Average intensity by decade
ax2.bar(decade_summary.index, decade_summary['mean'], color='darkred', alpha=0.7, label='Mean')
ax2.plot(decade_summary.index, decade_summary['max'], 'ko-', linewidth=2, markersize=8, label='Max')
ax2.set_title('Hurricane Intensity by Decade')
ax2.set_xlabel('Decade')
ax2.set_ylabel('Wind Speed (knots)')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nDecade Summary:")
print(decade_summary)

## Next Steps

This notebook has demonstrated:
1. Loading hurricane track data from HURDAT2
2. Exploring and visualizing hurricane tracks
3. Preprocessing and feature engineering
4. Data validation
5. Preparing sequences for model training

Next notebooks will cover:
- Model training with GraphCast and Pangu-Weather
- Implementing the CNN-Transformer hybrid model
- Ensemble forecasting
- Model evaluation and comparison with GFS/ECMWF